In [1]:
import pandas as pd
import altair as alt
from gprofiler import GProfiler

In [2]:
gp = GProfiler(return_dataframe=True)

In [3]:
delta_correlation_df = pd.read_csv('delta_correlation_df.csv')
delta_correlation_df

,Gene,Delta_Correlation,P_Value,FDR,Cancer
0,A1BG,-0.268533,5.703182e-02,1.320375e-01,CCRCC
1,A1CF,0.192038,1.063340e-04,6.401858e-04,CCRCC
2,A2M,-0.191619,1.277644e-01,2.439276e-01,CCRCC
3,AAAS,0.019654,8.963138e-01,9.409267e-01,CCRCC
4,AACS,-0.169937,6.007042e-02,1.375402e-01,CCRCC
5,AADAT,-0.263372,1.531939e-02,4.572896e-02,CCRCC
6,AAED1,0.190506,2.882440e-01,4.385777e-01,CCRCC
7,AAGAB,0.364999,7.633656e-04,3.646252e-03,CCRCC
8,AAK1,0.356932,6.347464e-14,1.385608e-12,CCRCC
9,AAMP,0.281272,2.536223e-02,6.920503e-02,CCRCC


In [4]:
def get_enriched_pathways(delta_corr, source):

    cancer_types = pd.unique(delta_corr.Cancer) # ['CCRCC', 'Endometrial', 'HNSCC', 'LSCC', 'LUAD']
    enriched_pathways = pd.DataFrame()
    for cancer in cancer_types:
        
        cancer_df = delta_corr[delta_corr.Cancer == cancer]
        background_genes  = list(pd.unique(cancer_df.Gene))
        sig_genes_df = cancer_df[cancer_df.FDR < 0.05]
        sig_genes  = list(pd.unique(sig_genes_df.Gene))

        cancer_profile = gp.profile(
            organism='hsapiens',
            query=sig_genes, 
            no_iea=True, 
            sources=[source],
            ordered=True,
            no_evidences=False,
            background=background_genes
        ).assign(cancer_type=cancer)
        
        enriched_pathways = enriched_pathways.append(cancer_profile)
    
    return enriched_pathways
        

In [5]:
enriched_kegg = get_enriched_pathways(delta_correlation_df, "KEGG")
enriched_wp = get_enriched_pathways(delta_correlation_df, "WP")
enriched_reactome = get_enriched_pathways(delta_correlation_df, "REAC")

## Which pathways are found in all cancers?

In [6]:
def summarize_pathways(all_profiles):
    
    # Group by pathway, to have an entry for which cancer types each pathway is enriched in
    # Then group by which cancer types each pathway is enriched in, so for each cancer type
    # group we have a list of all pathways that are enriched in all members of that group of
    # cancer types
    summ = all_profiles.\
    groupby("name").\
    agg(cancers_str=("cancer_type", lambda x: "_".join(sorted(set(x))))).\
    reset_index(drop=False).\
    groupby("cancers_str").\
    agg(pathways=("name", lambda x: sorted(set(x)))).\
    reset_index(drop=False)

    # Sort everything nicely
    summ.insert(0, "cancers", summ["cancers_str"].str.split("_"))
    summ = summ.\
    assign(cancers_ct=summ["cancers"].apply(len)).\
    sort_values(by=["cancers_ct", "cancers_str"], ascending=[False, True]).\
    drop(columns=["cancers_str", "cancers_ct"]).\
    reset_index(drop=True)

    summ = summ.\
    assign(pathways_ct=summ["pathways"].apply(len))

    return summ

In [7]:
kegg_pw_summ = summarize_pathways(enriched_kegg)
print(kegg_pw_summ["pathways_ct"].sum())
kegg_pw_summ

67


,cancers,pathways,pathways_ct
0,"[CCRCC, Endometrial, HNSCC, LSCC, LUAD]","[Alanine, aspartate and glutamate metabolism, ...",10
1,"[CCRCC, Endometrial, LSCC, LUAD]","[Arginine and proline metabolism, Butanoate me...",4
2,"[CCRCC, HNSCC, LSCC, LUAD]","[ABC transporters, Glycerolipid metabolism]",2
3,"[Endometrial, HNSCC, LSCC, LUAD]","[Fatty acid biosynthesis, Riboflavin metabolism]",2
4,"[CCRCC, Endometrial, LUAD]",[Biosynthesis of cofactors],1
5,"[CCRCC, HNSCC, LSCC]",[Vibrio cholerae infection],1
6,"[CCRCC, LSCC, LUAD]","[Carbon metabolism, Glyoxylate and dicarboxyla...",3
7,"[Endometrial, LSCC, LUAD]",[Thiamine metabolism],1
8,"[HNSCC, LSCC, LUAD]",[Nitrogen metabolism],1
9,"[CCRCC, HNSCC]","[Human immunodeficiency virus 1 infection, Oxi...",2


In [8]:
wp_pw_summ = summarize_pathways(enriched_wp)
print(wp_pw_summ["pathways_ct"].sum())
wp_pw_summ

70


,cancers,pathways,pathways_ct
0,"[CCRCC, Endometrial, HNSCC, LSCC, LUAD]",[Fatty Acid Biosynthesis],1
1,"[CCRCC, Endometrial, LSCC, LUAD]","[7-oxo-C and 7beta-HC pathways, Amino Acid met...",7
2,"[CCRCC, LSCC, LUAD]","[Ethanol effects on histone modifications, Fat...",6
3,"[Endometrial, LSCC, LUAD]","[Nanomaterial induced apoptosis, Prostaglandin...",2
4,"[HNSCC, LSCC, LUAD]","[DNA Damage Response, Integrated Cancer Pathwa...",3
5,"[CCRCC, HNSCC]","[Ebola Virus Pathway on Host, Mitochondrial co...",3
6,"[CCRCC, LUAD]","[Drug Induction of Bile Acid Pathway, Oxidativ...",2
7,"[HNSCC, LSCC]","[Apoptosis, Small cell lung cancer]",2
8,"[LSCC, LUAD]","[Apoptosis Modulation by HSP70, Benzo(a)pyrene...",8
9,[CCRCC],"[Butyrate-induced histone acetylation, Codeine...",9


## Heatmaps

Groups from https://academic.oup.com/database/article/doi/10.1093/database/baaa045/5857494

In [20]:
def pathway_heatmap(gp_results, reactome=False):
    
    if reactome:
        # Sort by hallmarks groups
        groups = pd.read_excel(
            "https://github.com/hocdb-KMU-TMU/hocdb/blob/master/Table%20S2.%20Hallmark%20of%20Cancer%20Data%20%20-%20Gene%20and%20Pathway.xlsx?raw=true",
            sheet_name="HP-pathway"
        )
        
        gp_results = gp_results.assign(react_id=gp_results["native"].str.split(":", expand=True)[1])
        
        gp_results = gp_results.merge(
            groups,
            left_on="react_id",
            right_on="REACTID",
            how="left"
        )
        
        gp_results = gp_results.assign(Hallmark=gp_results["Hallmark"].fillna("No hallmark"))
        
        charts = []
        for hallmark in gp_results["Hallmark"].unique():
            
            hallmark_results = gp_results[gp_results["Hallmark"] == hallmark]
            
            sort = hallmark_results.\
            groupby("name")["cancer_type"].\
            count().\
            sort_values(ascending=False).\
            index.\
            values

        
            charts.append(
                alt.Chart(hallmark_results).mark_rect().encode(
                    x=alt.X(
                        "cancer_type:N",
                    ),
                    y=alt.Y(
                        "name",
                        sort=sort,
                        axis=alt.Axis(
                            labelLimit=1000,
                            title=None,
                        )
                    ),
                    color=alt.Color(
                        "recall",
                        scale=alt.Scale(
                            scheme="reds"
                        )
                    ),
                ).properties(
                    title=hallmark,
                )
            )
        
        chart = alt.vconcat(*charts)

        
    else:
        # Sort based on how many cancers the pathway is enriched in
        sort = gp_results.\
        groupby("name")["cancer_type"].\
        count().\
        sort_values(ascending=False).\
        index.\
        values
    
        chart = alt.Chart(gp_results).mark_rect().encode(
            x=alt.X(
                "cancer_type:N",
            ),
            y=alt.Y(
                "name",
                sort=sort,
                axis=alt.Axis(
                    labelLimit=1000,
                    title=None,
                )
            ),
            color=alt.Color(
                "recall",
                scale=alt.Scale(
                    scheme="reds"
                )
            ),
        )

    return chart

In [12]:
pathway_heatmap(enriched_kegg)

alt.Chart(...)

In [13]:
pathway_heatmap(enriched_wp)

alt.Chart(...)

### Problem: We need better groups

In [14]:
groups = pd.read_excel(
    "https://github.com/hocdb-KMU-TMU/hocdb/blob/master/Table%20S2.%20Hallmark%20of%20Cancer%20Data%20%20-%20Gene%20and%20Pathway.xlsx?raw=true",
    sheet_name="HP-pathway"
)

In [15]:
groups.shape

(159, 6)

In [21]:
pathway_heatmap(enriched_reactome, reactome=True)

alt.VConcatChart(...)